# Scrapping Exercises - Part 2 - 2

You are now ready to get started to prepare your own spider. You have until next week to get review from London hotels ;) ! Remember, spiders are python files, but using jupyter notebook can be very useful to try and test.

In case of need, here is the [scrapy documentation](https://docs.scrapy.org/en/latest/).

Here is how you could proceed :
- Open a review page and discover how to parse it
- Open a restaurant page and discover how to get reviews urls
- Open a restaurant page and discover how to get to next page of reviews
- Open a city page and discover how to get restaurant urls
- Open a city page and discover how to get the net page of restaurants
- Try to put it altogether
- Be careful, there a lots of restaurants with a lot of comments

But of course you are free to proceed as you want to !

_NB : To begin, you might want to get only short-description of reviews from restaurant pages and not total review from review pages ;) ! Might be easier to begin !_


## Making import

In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [3]:
import os
os.chdir("drive/MyDrive/NLP @ X_HEC - 2K21/Cours 1 - Scrapping/")

In [ ]:
!pip install -r requirements.txt

In [5]:
# Imports
import json
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import logzero
import scrapy



import requests
from scrapy.selector import Selector

In [6]:
# Parameters for spider execution
data_directory_path = 'scrapped_data/'
file_name = 'trip_data.jl'
spider_name = 'RestoReviewSpider'
file_path_spider = "scrapped_data/"+file_name

os.chdir("scrapy_project/")
!pwd
!ls

/content/drive/MyDrive/NLP @ X_HEC - 2K21/Cours 1 - Scrapping/scrapy_project
scrapped_data  scrapy.cfg  TA_scrapy


### Lauching the spyder

In [24]:
!scrapy crawl RestoReviewSpider -o scrapped_data/trip_data.jl 

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
[W 210129 11:32:45 restoSpiderReview_simplified:152]  rep({'Restaurant': ' Frog & Underground ', 'Adresse': '176 rue Montmartre', 'Rank': '#20', 'Date': '20 November 2019', 'Tittle': '“Affordable good food and an incredible service!”\n', 'Review': 'The Food was good the prices were not too bad but big thanks to Cecilie and Renata that were happy to help and amazingly quick!', 'Grade': '4', 'Username': 'Selim B', 'Price': ' US$15 - US$23 ', 'Style': []})
[W 210129 11:32:45 restoSpiderReview_simplified:106]  > PARSING NEW REVIEW NUMBER (17315)
[W 210129 11:32:45 restoSpiderReview_simplified:107]  Response (<200 https://www.tripadvisor.co.uk/ShowUserReviews-g187147-d10449920-r728245974-Frog_Underground-Paris_Ile_de_France.html>)
[W 210129 11:32:45 restoSpiderReview_simplified:152]  rep({'Restaurant': ' Frog & Underground ', 'Adresse': '176 rue Montmartre', 'Rank': '#20', 'Date': '22 November 2019', 'Tittle': '“W

### Have a look at our review

In [7]:
# Function rto read jl file
def read_jl_file(file_name):
    values = []
    with open(file_name, 'rb') as f:
        line = '---'
        while len(line)>1:
            line = f.readline()
            values.append(line)
    values = values[:-1]
    values = [json.loads(i) for i in values]
    df = pd.DataFrame(values)
    return df

df = read_jl_file(data_directory_path+file_name)

# Having a brief look at the data shape
df.shape

(18920, 10)

In [8]:
df.head()

,Restaurant,Adresse,Rank,Date,Tittle,Review,Grade,Username,Price,Style
0,Epicure,112 Rue Du Faubourg Saint-Honore,#14,12 March 2020,“Mikey speaks”\n,"Valentines Day at the Epicure! champagne, truf...",5,parksloper511,,[]
1,Epicure,112 Rue Du Faubourg Saint-Honore,#14,9 March 2020,“Amazing”\n,"Had an amazing experience - incredible food, a...",5,MsNinaG,,[]
2,Epicure,112 Rue Du Faubourg Saint-Honore,#14,22 October 2019,“Amazing!”\n,"The service, atmosphere and food were amazing....",5,HarmonyLiu,,[]
3,Epicure,112 Rue Du Faubourg Saint-Honore,#14,23 October 2019,“I enjoyed the best meal of my life here at Ep...,"I'm going to say that this is, by some great m...",5,theglobalexplorer1,,[]
4,Domenico's,8 rue Saint Lazare,#13,31 January 2020,“Domenico’s is the best italian”\n,"Domenico’s is the best italian I ever had, all...",4,narjissj2016,US$16 - US$22,[]


Our dataset is composed of several features : 

*   **Restaurant** : the name of the restaurant
*   **Adresse** : the adress of the restaurant
*   **Rank** : all the restaurant in Tripadvisor are ranked and we just extracted this rank.
*   **Tittle** : the tittle of the review
*   **Review** : the complete review
*   **Grade** : the global grade of the restaurant
*   **Username** : the username of the person who wrote the review
*   **Price** : the price range of the restaurant in dollar
*   **Style** : the cuisine style of the restaurant (ex : French, Italian...)

Let's notice that those two last column are often empty, it is linked to the fact that those information are not complete/available on tripadvisor.


In [9]:
#Let's have a look at some random review
df['Review'][2334]

"Called on Monday night on our way back to our hotel for a nightcap, and found the welcome to be so warm and friendly. We booked a table for dinner the next night. Fantastic menu every item is hand picked and there's a great selection of wines. The food definitely lived up to our expectations, and is all reasonablypriced. So if you're in Paris and looking for a bite to eat you have to try the Publisher you'll be glad you did."

In [13]:
#Let's see how many estaurant do we have and how many review per restaurant
print("The mean of restaurant per review is :", df['Restaurant'].value_counts().mean())
print(df['Restaurant'].value_counts())

The mean of restaurant per review is : 157.66666666666666
 Sacree fleur                  2057
 Epicure                       1210
 Le Cinq                       1105
 Les Apotres de Pigalle         780
 Il Etait Un Square             673
                               ... 
 Tra di Noi Vasco De Gama         8
 La Table d'Hami                  7
 Oinari Paris                     4
 La Cantine de Meme               3
 Le Faham by Kelly Rangama        2
Name: Restaurant, Length: 120, dtype: int64


Let's recall first that we used the review from Paris restaurant to build this dataset. We have a total of 120 restaurant, nevertheless the number of review per restaurant is quite 
heterogeneous. We only scraped the review in english for the 120th first restaurant in Paris (first ranked by tripadvisor on their webste), which explain why we have some restaurant with only a few reviews. 